# **Segmentez des clients d'un site e-commerce**

# **Contexte**
Olist souhaite qu'on fournit à ses équipes d'e-commerce une segmentation des clients qu’elles pourront utiliser au quotidien pour leurs campagnes de communication.
L'objectif de ce notebook est de comprendre les différents types d’utilisateurs grâce à leur comportement et à leurs données personnelles.

# **Plan du notebook**
1. Import des données
2. Nettoyage des données
3. Analyse des données 

### **1. Import des données**

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
customers = pd.read_csv('../input/segmentation/olist_customers_dataset.csv')
geolocation = pd.read_csv('../input/segmentation/olist_geolocation_dataset.csv')
order_items = pd.read_csv('../input/segmentation/olist_order_items_dataset.csv')
order_payments = pd.read_csv('../input/segmentation/olist_order_payments_dataset.csv')
orders = pd.read_csv('../input/segmentation/olist_orders_dataset.csv')
products = pd.read_csv('../input/segmentation/olist_products_dataset.csv')
sellers = pd.read_csv('../input/segmentation/olist_sellers_dataset.csv')
product_category_name_translation = pd.read_csv('../input/segmentation/product_category_name_translation.csv')
reviews_dataset = pd.read_csv('../input/segmentation/olist_order_reviews_dataset.csv')

### **2.Nettoyage des données**

Exploration des dataframes

In [3]:
orders.head()

In [4]:
customers.head()

In [5]:
products.head()

In [6]:
reviews_dataset.head()

In [7]:
#trying to remove outliers before EDA

In [8]:
price = order_items['price']

In [9]:
import plotly.express as px

fig = px.box(price)


print('mean : ',np.round(price.mean()),2)
print('std  : ',np.round(price.std()),2)

fig.update_traces(marker_color='#264653', opacity=.7)
fig.update_layout(title_text='boxplot of price before removing outliers')
fig.show()

In [10]:
d = order_items['price']

d = d.mask((d-d.mean()).abs() > .7 * d.std()  )

fig = px.box(d)


print('mean : ' ,np.round(d.mean()),2)
print('std  : ',np.round(d.std(),2))

fig.update_traces(marker_color='#264653', opacity=.7)
fig.update_layout(title_text='boxplot of price after removing outliers')
fig.show()

In [11]:
order_items['price']=d

In [12]:
order_items.columns

In [13]:
orders.order_id.value_counts()

In [14]:
reviews_dataset.head()

In [15]:
orders.head()

In [16]:
orders.sort_values(by='order_purchase_timestamp', ascending=False).head(5)

In [17]:
# 1ère date 04/09/2016
# dernière date 17/10/2018

In [18]:
orders.sort_values(by='order_delivered_customer_date', ascending=False).tail(30)
orders.groupby('order_status').sum()

In [19]:
orders.order_status.isin(['canceled','unavailable']).sum()


In [20]:
orders.order_status.isin(['shipped','delivered']).sum()

In [21]:
orders.shape

In [22]:
orders[orders.customer_id.duplicated()]
orders[orders.duplicated(['customer_id'])]

Construire un dictionnaire pour associer à chaque client un seul et unique id

In [23]:
d = dict(zip(customers.customer_id, customers.customer_unique_id))
d

In [24]:
#df5.customer_unique_id.value_counts().to_dict()

In [25]:
reviews_dataset.head()

In [26]:
reviews_dataset.order_id.value_counts()

Par une aggrégation, on calcule ici le review_score par commande.

In [27]:
reviews_dataset = reviews_dataset.groupby(['order_id']).agg({
        'review_score': 'mean'})

In [28]:
reviews_dataset.review_score.value_counts().plot(kind='pie',autopct='%1.1f%%')

In [29]:
orders.customer_id = orders.customer_id.map(d)

In [30]:
orders.customer_id.value_counts()

**Faire le merge des différents dataframes**

In [31]:
# joindre orders et order items
df1 = pd.merge(orders, order_items, on='order_id')
df1 = pd.merge(df1,reviews_dataset,on='order_id')
# joindre df1 et customers (changed to unique_id)
df1 = df1.rename(columns={'customer_id':'customer_unique_id'})
#customers.product_category_name = df5.product_category_name.map(category_dict)

df2 = pd.merge(df1, customers, on='customer_unique_id')

In [32]:
df1.columns

In [33]:
# joindre df2 et seller
df3 = pd.merge(df2, sellers, on='seller_id')

In [34]:
# joindre df3 et products
df4= pd.merge(df3, products, on='product_id')

In [35]:
df4.head()

In [36]:
df5 = pd.merge(df4,order_payments,on='order_id')

In [37]:
df5.corr()

In [38]:
customers.head()

In [39]:
customers.isnull().sum()
customers.shape

In [40]:
customers.customer_state.value_counts()

In [41]:
customers.customer_state.value_counts()
# ça compte 27 states

In [42]:
sellers.head()

In [43]:
sellers.seller_zip_code_prefix.value_counts()

In [44]:
order_items.head()

In [45]:
order_items.shape

In [46]:
order_items.isnull().sum()

In [47]:
reviews_dataset.head()

In [48]:
products.head()

In [49]:
products.product_category_name.value_counts()

In [50]:
df5.product_category_name.value_counts()

In [51]:
df5.product_category_name.isnull().sum()

In [52]:
products.product_category_name.isnull().sum()

In [53]:
products.columns

In [54]:
df5.shape

In [55]:
product_category_name_translation.head()

In [56]:
category_dict = pd.Series(product_category_name_translation.product_category_name_english.values,index=product_category_name_translation.product_category_name).to_dict()

Construire un dictionnaire category_dict pour traduire les catégories en portugais dans notre df5 en des catégories écrites en anglais. 

In [57]:
category_dict

In [58]:
df5.product_category_name = df5.product_category_name.map(category_dict)


Groupement des catégories en des plus grandes catégories pour faciliter l'encodage de cette variable après.

In [59]:
product_categories_dict = {
    'construction_tools_construction': 'construction',
    'construction_tools_lights': 'construction',
    'construction_tools_safety': 'construction',
    'costruction_tools_garden': 'construction',
    'costruction_tools_tools': 'construction',
    'garden_tools': 'construction',
    'home_construction': 'construction',

    'fashio_female_clothing': 'fashion',
    'fashion_bags_accessories': 'fashion',
    'fashion_childrens_clothes': 'fashion',
    'fashion_male_clothing': 'fashion',
    'fashion_shoes': 'fashion',
    'fashion_sport': 'fashion',
    'fashion_underwear_beach': 'fashion',

    'furniture_bedroom': 'furniture',
    'furniture_decor': 'furniture',
    'furniture_living_room': 'furniture',
    'furniture_mattress_and_upholstery': 'furniture',
    'bed_bath_table': 'furniture',
    'kitchen_dining_laundry_garden_furniture': 'furniture',
    'office_furniture': 'furniture',

    'home_appliances': 'home',
    'home_appliances_2': 'home',
    'home_comfort_2': 'home',
    'home_confort': 'home',
    'air_conditioning': 'home',
    'housewares': 'home',
    'art': 'home',
    'arts_and_craftmanship': 'home',
    'flowers': 'home',
    'cool_stuff': 'home',

    'drinks': 'food_drink',
    'food': 'food_drink',
    'food_drink': 'food_drink',
    'la_cuisine': 'food_drink',
    'electronics': 'electronics',
    'audio': 'electronics',
    'tablets_printing_image': 'electronics',
    'telephony': 'electronics',
    'fixed_telephony': 'electronics',
    'small_appliances': 'electronics',
    'small_appliances_home_oven_and_coffee': 'electronics',
    'computers_accessories': 'electronics',
    'computers': 'electronics',
'sports_leisure': 'sports_leisure',
    'consoles_games': 'sports_leisure',
    'musical_instruments': 'sports_leisure',
    'toys': 'sports_leisure',
    'cine_photo': 'sports_leisure',
    'dvds_blu_ray': 'sports_leisure',
    'cds_dvds_musicals': 'sports_leisure',
    'music': 'sports_leisure',
    'books_general_interest': 'sports_leisure',
    'books_imported': 'sports_leisure',
    'books_technical': 'sports_leisure',

    'health_beauty': 'health_beauty',
    'perfumery': 'health_beauty',
    'diapers_and_hygiene': 'health_beauty',
    'baby': 'health_beauty',

    'christmas_supplies': 'supplies',
    'stationery': 'supplies',
    'party_supplies': 'supplies',
    'auto': 'supplies',
    'luggage_accessories': 'supplies',

    'watches_gifts': 'gifts',

    'agro_industry_and_commerce': 'misc',
    'industry_commerce_and_business': 'misc',
    'security_and_services': 'misc',
    'signaling_and_security': 'misc',
    'market_place': 'misc',
    'pet_shop': 'misc',
}

In [60]:
df5.product_category_name.value_counts()

In [61]:
df5.product_category_name = df5.product_category_name.map(product_categories_dict)


In [62]:
df5.product_category_name.value_counts()

In [63]:
df5[['order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date','order_purchase_timestamp','shipping_limit_date']] =df5[['order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date','order_purchase_timestamp','shipping_limit_date']].apply(pd.to_datetime)


In [64]:
df5.dtypes

In [65]:
cat = df5.select_dtypes(include='O').keys()
dates = df5.select_dtypes(include='datetime64[ns]').keys()

In [66]:
#df5.isnull().sum()

In [67]:
df5.payment_type.value_counts()

In [68]:
df5.order_status.value_counts()

Groupement des villes des clients en 5 régions

In [69]:
customers.customer_state.value_counts()
#divisons les états du brésil selon leurs localisation
#régions 1-nord 2-nord est 3-centre-est 4-sud-est 5-sud
# 1-AM,AC,TO,RO,PA,RR,
# 2-PE,AL,BA,CE,MA,PB,PI,RN,SE,
# 3-GO,DF,MT,MS
# 4- SP,MG,ES,RJ,
# 5- SC,PR,RS,

In [70]:
states_dict=dict.fromkeys(['AM','AC','TO','RO','PA','RR'] ,'north')
states_dict.update(dict.fromkeys(['PE','AL','BA','CE','MA','PB','PI','RN','SE'], 'north east'))
states_dict.update(dict.fromkeys(['GO','DF','MT','MS'], 'center east'))
states_dict.update(dict.fromkeys(['SP','MG','ES','RJ'], 'south east'))
states_dict.update(dict.fromkeys(['SC','PR','RS'], 'south'))

In [71]:
states_dict

In [72]:
df5.seller_state = df5.seller_state.map(states_dict)
df5.customer_state = df5.customer_state.map(states_dict)

In [73]:
df5.columns

In [74]:
df5.product_category_name.value_counts().plot(kind='pie',ylabel='')
plt.title("product_category_name")

In [75]:
df5.payment_type.value_counts().plot(kind='pie')

In [76]:
df5.head()

Encodage des variables catégorielles

In [77]:
import pandas as pd

# Multiple categorical columns

df5 = pd.get_dummies(df5, columns=['order_status','payment_type','product_category_name','customer_state','seller_state'])

In [78]:
df5

In [79]:
df5.columns

In [80]:
cat.difference(['payment_type','order_status','product_category_name'])

In [81]:
orders.customer_id.isnull().sum()

In [82]:
df5.order_delivered_customer_date.describe()

In [83]:
df6 = df5[df5.order_status_delivered == 1]

Ne garder que les commandes qui sont délivrées et supprimer ceux qui sont dupliquées

In [84]:
df6 = df6.drop(df6[df6.order_id.duplicated()== True].index)

**EDA**

In [85]:
from datetime import datetime

order_items['shipping_limit_month'] = order_items['shipping_limit_date'].apply(lambda x:datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%B-%Y'))

bar = order_items['shipping_limit_month'].value_counts()

In [86]:
fig = px.bar(x=bar.index , y=bar.values , text=bar.values , labels={'x':'Months','y':'Count'})

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside',
                 marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                 marker_line_width=1.5, opacity=.7)

fig.update_layout(title_text='Months that have Top shipping')
fig.show()

In [87]:
fig = px.histogram(x=order_items['shipping_limit_month'], y=order_items['price'],
                  labels={'x':'Months','y':'price'})


fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.7)


fig.update_layout(title_text='Sum of Total price in each Month')

In [88]:
df5.head()

In [89]:
orders.order_status.isin(['canceled','unavailable']).sum()
#on peut supprimer ces lignes puisque pas de d'achat finalement non ?

In [90]:
orders.order_status.isin(['shipped','delivered']).sum()

Calculer le nombre total des commandes/client par cette aggrégation

In [91]:
fréquence = df5.groupby(['customer_unique_id']).agg({
        'order_id': 'count',
        })

In [92]:
fréquence[fréquence.order_id == 272]

In [93]:
fréquence.order_id.value_counts()

In [94]:
labels= ['1 commande','2 commandes','3 commandes','4 commandes']
plt.pie(fréquence.order_id.value_counts())
plt.legend(labels=labels)
plt.title("la fréquence des commandes par client")


In [95]:
reviews_dataset.review_score.value_counts().plot(kind='pie',autopct='%1.1f%%')

A partir de ce pie chart, on la majorité des clients sont satisfaits.

In [96]:
customers.customer_state.value_counts()[:10]

In [97]:
customers.customer_state.value_counts().plot(kind='bar')
plt.title('les villes des clients')

A partir de ce graph, on constate que la majorité des clients viennent du sud_est et du sud du brésil.

In [98]:
sellers.seller_state.value_counts().plot(kind='bar')
plt.title('les villes des vendeurs')

In [99]:
df5.corr()

In [100]:
df6.describe()

In [101]:
#export du résultat
df6.to_csv('df6.csv')